# RusT Petalo

In [1]:
using Pkg
Pkg.activate(normpath(joinpath(@__DIR__, "..")))  #activate environment in ,,/src

In [2]:
using Logging

In [3]:
using Plots
using DataFrames
using HDF5
using PhysicalConstants
using PhysicalConstants.CODATA2018
using Unitful
using Glob
using CSV
using Printf

In [4]:
using ATools

In [66]:
include("../src/ANema.jl") 

Main.ANema

In [6]:
import Unitful:
    nm, μm, mm, cm, ns, μs, ms, ps, s

In [3]:
function loglevel(log)
    if log == "Debug"
        logger = SimpleLogger(stdout, Logging.Debug)
    elseif log =="Info"
        logger = SimpleLogger(stdout, Logging.Info)
    else
        logger = SimpleLogger(stdout, Logging.Warn)
    end
    old_logger = global_logger(logger)
end

loglevel (generic function with 1 method)

In [4]:
loglevel("Warn")

Base.CoreLogging.SimpleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(48) open, 0 bytes waiting))), Info, Dict{Any, Int64}())

## Run cargo

## Read mdf files

In [ ]:
#conf="n3-w-20mm-all"

In [25]:
function rcmd(conf; 
              excm      = false, 
              slices    = (101,501,251), 
              bins      = (101,501,251), 
              estimator = "zstd", 
              reco      = "true-first", 
              rdir      = "/Users/jj/JuliaProjects/ANema/data/",
              head      = "cargo run --features serial --release --bin mlem --",
              dst       = "--dataset true_info/lors")
    
    ldir = string(conf,"-lor")
    idir = joinpath(rdir, ldir)
    pfx  = string("-",estimator,"-",conf,"-",reco)
    on   = string("mlem",pfx)
    ofn = joinpath(rdir,on,"img") 
    slc  = "-s $(slices[1]),$(slices[2]),$(slices[3])"
    bns  = "-n $(bins[1]),$(bins[2]),$(bins[3])"
    fn   = string("lor", pfx,".h5")
    #idir = "/Users/jj/JuliaProjects/ANema/data/n3-w-20mm-all-lor/"
    #fn   = "lor-zstd-n3-w-20mm-all-average-recall.h5"
    #ofn  =  "/Users/jj/JuliaProjects/ANema/data/mlem-zstd-n3-w-20mm-all-average-recall/img"
    path = joinpath(idir, fn)
    in   = "-f $path"
    out  = "--out-files $ofn"
    cmd  = "$head $slc $bns $dst $in $out"
    if excm
        println("executing: $cmd")
        run(`$cmd`)
    else
        return cmd
    end 
end

rcmd (generic function with 4 methods)

In [29]:
rcmd("n3-w-40mm-all-qc3", excm = false, estimator = "zstd", reco = "average-recall")

"cargo run --features serial --release --bin mlem -- -s 101,501,251 -n 101,501,251 --dataset true_info/lors -f /Users/jj/JuliaProjects/ANema/data/n3-w-40mm-all-qc3-lor/lor-zstd-n3-w-40mm-all-qc3-average-recall.h5 --out-files /Users/jj/JuliaProjects/ANema/data/mlem-zstd-n3-w-40mm-all-qc3-average-recall/img"

In [24]:
rcmd("n3-w-20mm-all",  estimator = "zstd", reco = "true-first")

"cargo run --features serial --release --bin mlem -- -s (101, 501, 251) -n (101, 501, 251) --dataset true_info/lors -f /Users/jj/JuliaProjects/ANema/data/n3-w-20mm-all-lor/lor-zstd-n3-w-20mm-all-true-first.h5 --out-files /Users/jj/JuliaProjects/ANema/data/mlem-zstd-n3-w-20mm-all-true-first/img"

In [57]:
n3p = DataFrame(CSV.File(pathf.phipath))
n3z = DataFrame(CSV.File(pathf.zpath));


In [58]:
println(names(n3p))

["nsipm1", "nsipm2", "phistd1", "phistd2", "q1", "q2", "r1", "r2", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zstd1", "zstd2", "zt1", "zt2", "r1x", "r2x"]


In [59]:
println(names(n3z))

["nsipm1", "nsipm2", "phistd1", "phistd2", "q1", "q2", "r1", "r2", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zstd1", "zstd2", "zt1", "zt2", "r1x", "r2x"]


In [60]:
n3mp = ANema.setunits(n3p)
n3mz = ANema.setunits(n3z)

println("phi data frame has length of ", nrow(n3mp))
println("z data frame has length of ", nrow(n3mz))
println("phi data frame names ", names(n3mp))
println("z data frame names ", names(n3mz))

phi data frame has length of 216227
z data frame has length of 266962
phi data frame names ["nsipm1", "nsipm2", "phistd1", "phistd2", "zstd1", "zstd2", "q1", "q2", "r1", "r1x", "r2", "r2x", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zt1", "zt2"]
z data frame names ["nsipm1", "nsipm2", "phistd1", "phistd2", "zstd1", "zstd2", "q1", "q2", "r1", "r1x", "r2", "r2x", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zt1", "zt2"]


## Write LORS

In [35]:
lors_first_true = ANema.dftolor(n3mp, ANema.dtfirst, ANema.postrue);

In [51]:
getlorpath(pathf.lordir,pathf.philp, "first-true")

"../data/n3-w-20mm-all-lor/lor-phistd-n3-w-20mm-all-first-true.h5"

In [52]:

ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "first-true"), lors_first_true)

In [53]:
lors_first_reco = ANema.dftolor(n3mp, ANema.dtfirst, ANema.posreco)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "first-reco"), lors_first_reco)

In [54]:
lors_minimum_reco = ANema.dftolor(n3mp, ANema.dtminimum, ANema.posreco)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "minimum-reco"), lors_minimum_reco)

In [62]:
n3mp.r2x[1:2]

2-element Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(mm,), 𝐋, nothing}}}:
  366.1171080961141 mm
 367.33644496451706 mm

In [67]:
lors_minimum_recall = ANema.dftolor(n3mp, ANema.dtminimum, ANema.posrecall)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "minimum-recall"), lors_minimum_recall)

In [68]:
lors_average_recall = ANema.dftolor(n3mp, ANema.dtaverage, ANema.posrecall)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.philp, "average-recall"), lors_average_recall)

In [70]:
lors_first_true = ANema.dftolor(n3mz, ANema.dtfirst, ANema.postrue)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "first-true"), lors_first_true)
lors_first_reco = ANema.dftolor(n3mz, ANema.dtfirst, ANema.posreco)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "first-reco"), lors_first_reco)
lors_minimum_reco = ANema.dftolor(n3mz, ANema.dtminimum, ANema.posreco)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "minimum-reco"), lors_minimum_reco)
lors_minimum_recall = ANema.dftolor(n3mz, ANema.dtminimum, ANema.posrecall)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "minimum-recall"), lors_minimum_recall)
lors_average_recall = ANema.dftolor(n3mz, ANema.dtaverage, ANema.posrecall)
ANema.write_lors_hdf5(getlorpath(pathf.lordir,pathf.zlp, "average-recall"), lors_average_recall)